# Install Libraries

In [1]:
!pip install dask[dataframe] protobuf==3.20.* -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.6 MB/s eta 0:00:00


In [2]:
#Install the necessary libraries for forecasting
!pip install statsforecast mlforecast neuralforecast nixtla -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.5/273.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Importing Libraries Required for this Notebook

In [3]:
from nixtla import NixtlaClient
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS, AutoARIMA, AutoETS
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from mlforecast.target_transforms import AutoDifferences
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT, NHITS
from neuralforecast.losses.pytorch import MAE
from nixtla import NixtlaClient
from functools import reduce
#import timesfm

# Data Preparation

In [4]:
data = pd.read_excel("monthly_data.xlsx")

In [5]:
data.head()

date warehouse  orders                    holiday_name  holiday  \
0 2020-12-01  Prague_1  190208  Christmas Eve2nd Christmas Day        2   
1 2021-01-01  Prague_1  235089                   New Years Day        1   
2 2021-02-01  Prague_1  222102                               0        0   
3 2021-03-01  Prague_1  254258        International womens day        0   
4 2021-04-01  Prague_1  242598        Good FridayEaster Monday        4   

   shutdown  mini_shutdown  shops_closed  winter_school_holidays  \
0         0              0             1                       0   
1         0              0             1                       0   
2         0              0             0                       0   
3         0              0             0                      14   
4         0              0             1                       0   

   school_holidays  blackout  mov_change  frankfurt_shutdown  precipitation  \
0                0         0           0                   0            8.3   
1                0         0           0                   0          166.3   
2                0         0           0                   0           37.0   
3                0         0           0                   0           24.9   
4                0         0           0                   0           13.4   

    snow  user_activity_1  user_activity_2  \
0    0.0            45470           839751   
1  112.0            53790           996212   
2  134.0            49832           943849   
3    3.0            55508          1094886   
4    2.0            50988          1041067   

                                                  id  
0  Prague_1_2020-12-05Prague_1_2020-12-06Prague_1...  
1  Prague_1_2021-01-01Prague_1_2021-01-02Prague_1...  
2  Prague_1_2021-02-01Prague_1_2021-02-02Prague_1...  
3  Prague_1_2021-03-01Prague_1_2021-03-02Prague_1...  
4  Prague_1_2021-04-01Prague_1_2021-04-02Prague_1...

In [6]:
data.columns

Index(['date', 'warehouse', 'orders', 'holiday_name', 'holiday', 'shutdown',
       'mini_shutdown', 'shops_closed', 'winter_school_holidays',
       'school_holidays', 'blackout', 'mov_change', 'frankfurt_shutdown',
       'precipitation', 'snow', 'user_activity_1', 'user_activity_2', 'id'],
      dtype='object')

## Dropping the columns that wont be known in advance.

In [7]:
data = data.drop(["holiday_name", 'shutdown', 'mini_shutdown', 'blackout', 'frankfurt_shutdown', "id", 'blackout', 'mov_change', 'frankfurt_shutdown','precipitation', 'snow', 'user_activity_1', 'user_activity_2'], axis=1)

In [8]:
#rename as per requirement of the forecast libraries
data = data.rename(columns={"warehouse": "unique_id", "date": "ds", "orders": "y"})

In [9]:
data.head()

ds unique_id       y  holiday  shops_closed  winter_school_holidays  \
0 2020-12-01  Prague_1  190208        2             1                       0   
1 2021-01-01  Prague_1  235089        1             1                       0   
2 2021-02-01  Prague_1  222102        0             0                       0   
3 2021-03-01  Prague_1  254258        0             0                      14   
4 2021-04-01  Prague_1  242598        4             1                       0   

   school_holidays  
0                0  
1                0  
2                0  
3                0  
4                0

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ds                      160 non-null    datetime64[ns]
 1   unique_id               160 non-null    object        
 2   y                       160 non-null    int64         
 3   holiday                 160 non-null    int64         
 4   shops_closed            160 non-null    int64         
 5   winter_school_holidays  160 non-null    int64         
 6   school_holidays         160 non-null    int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 8.9+ KB


In [11]:
data['ds'] = pd.to_datetime(data['ds'])

In [12]:
#Splitting the Data for Trasin & Test
train = data[data['ds'] <= '2023-11-01']
test = data[data['ds'] > '2023-11-01']

In [13]:
train.groupby('unique_id')['y'].count()

unique_id
Brno_1      36
Prague_1    36
Prague_2    36
Prague_3    36
Name: y, dtype: int64

In [14]:
train.head()

ds unique_id       y  holiday  shops_closed  winter_school_holidays  \
0 2020-12-01  Prague_1  190208        2             1                       0   
1 2021-01-01  Prague_1  235089        1             1                       0   
2 2021-02-01  Prague_1  222102        0             0                       0   
3 2021-03-01  Prague_1  254258        0             0                      14   
4 2021-04-01  Prague_1  242598        4             1                       0   

   school_holidays  
0                0  
1                0  
2                0  
3                0  
4                0

# SFFORECAST

###Defining Statforecast Parameters

In [16]:
models = [AutoARIMA(season_length=12)]

sf = StatsForecast(models=models, freq='MS', n_jobs=1)

In [18]:
test_ = test.drop("y", axis=1)

In [19]:
horizon = 4
sf_fcst = sf.forecast(df=train, h=horizon, X_df=test_,level=(5,10,20,30,40,50,60,70,80,90,95))

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [20]:
sf_fcst.head()

ds      AutoARIMA  AutoARIMA-lo-95  AutoARIMA-lo-90  \
unique_id                                                               
Brno_1    2023-12-01  258943.875000     2.336649e+05     2.377290e+05   
Brno_1    2024-01-01  260850.796875     2.251009e+05     2.308485e+05   
Brno_1    2024-02-01  255345.437500     2.115609e+05     2.186003e+05   
Brno_1    2024-03-01  255379.921875     2.048219e+05     2.129503e+05   
Prague_1  2023-12-01  300422.125000    -7.544180e+07    -6.326446e+07   

           AutoARIMA-lo-80  AutoARIMA-lo-70  AutoARIMA-lo-60  AutoARIMA-lo-50  \
unique_id                                                                       
Brno_1        2.424148e+05     2.455763e+05     2.480889e+05     2.502445e+05   
Brno_1        2.374752e+05     2.419462e+05     2.454995e+05     2.485480e+05   
Brno_1        2.267163e+05     2.321921e+05     2.365441e+05     2.402777e+05   
Brno_1        2.223218e+05     2.286447e+05     2.336700e+05     2.379812e+05   
Prague_1     -4.922476e+07    -3.975224e+07    -3.222378e+07    -2.576503e+07   

           AutoARIMA-lo-40  AutoARIMA-lo-30  ...  AutoARIMA-hi-10  \
unique_id                                    ...                    
Brno_1        2.521803e+05     2.539741e+05  ...     2.605646e+05   
Brno_1        2.512857e+05     2.538225e+05  ...     2.631429e+05   
Brno_1        2.436306e+05     2.467376e+05  ...     2.581526e+05   
Brno_1        2.418528e+05     2.454404e+05  ...     2.586214e+05   
Prague_1     -1.996488e+07    -1.459017e+07  ...     5.156568e+06   

           AutoARIMA-hi-20  AutoARIMA-hi-30  AutoARIMA-hi-40  AutoARIMA-hi-50  \
unique_id                                                                       
Brno_1        2.622115e+05     2.639136e+05     2.657074e+05     2.676432e+05   
Brno_1        2.654719e+05     2.678791e+05     2.704159e+05     2.731536e+05   
Brno_1        2.610051e+05     2.639533e+05     2.670603e+05     2.704132e+05   
Brno_1        2.619151e+05     2.653194e+05     2.689070e+05     2.727787e+05   
Prague_1      1.009094e+07     1.519102e+07     2.056572e+07     2.636588e+07   

           AutoARIMA-hi-60  AutoARIMA-hi-70  AutoARIMA-hi-80  AutoARIMA-hi-90  \
unique_id                                                                       
Brno_1        2.697988e+05     2.723115e+05     2.754729e+05     2.801587e+05   
Brno_1        2.762021e+05     2.797554e+05     2.842264e+05     2.908531e+05   
Brno_1        2.741468e+05     2.784988e+05     2.839746e+05     2.920906e+05   
Brno_1        2.770899e+05     2.821151e+05     2.884381e+05     2.978096e+05   
Prague_1      3.282462e+07     4.035308e+07     4.982560e+07     6.386530e+07   

           AutoARIMA-hi-95  
unique_id                   
Brno_1        2.842229e+05  
Brno_1        2.966008e+05  
Brno_1        2.991300e+05  
Brno_1        3.059380e+05  
Prague_1      7.604265e+07  

[5 rows x 24 columns]

### Defining AutoTBATS & AutoETS

In [21]:
models = [AutoTBATS(season_length=12),AutoETS(season_length=12)]

In [22]:
stat_fcst = StatsForecast(models=models,freq='MS')

In [23]:
train_stat_df_ETS_TBATS = train[["unique_id", "ds", "y"]]

In [24]:
ets_tbats = stat_fcst.forecast(df=train_stat_df_ETS_TBATS, h=4,level=(5,10,20,30,40,50,60,70,80,90,95))

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [25]:
ets_tbats.head()

ds      AutoTBATS  AutoTBATS-lo-5  AutoTBATS-lo-10  \
unique_id                                                              
Brno_1    2023-12-01  249276.234375   248604.968750    247931.046875   
Brno_1    2024-01-01  249350.562500   248634.031250    247914.687500   
Brno_1    2024-02-01  246904.156250   246154.921875    245402.718750   
Brno_1    2024-03-01  242299.828125   241526.640625    240750.390625   
Prague_1  2023-12-01  286205.500000   285441.437500    284674.343750   

           AutoTBATS-lo-20  AutoTBATS-lo-30  AutoTBATS-lo-40  AutoTBATS-lo-50  \
unique_id                                                                       
Brno_1       246564.203125    245151.437500    243662.609375    242055.937500   
Brno_1       246455.671875    244947.671875    243358.453125    241643.453125   
Brno_1       243877.078125    242300.218750    240638.453125    238845.140625   
Brno_1       239176.015625    237548.750000    235833.875000    233983.250000   
Prague_1     283118.531250    281510.437500    279815.781250    277987.000000   

           AutoTBATS-lo-60  AutoTBATS-lo-70  ...  AutoETS-hi-10  \
unique_id                                    ...                  
Brno_1       240266.812500    238181.375000  ...  253588.203125   
Brno_1       239733.718750    237507.671875  ...  254111.500000   
Brno_1       236848.203125    234520.515625  ...  254531.765625   
Brno_1       231922.484375    229520.421875  ...  254893.093750   
Prague_1     275950.531250    273576.781250  ...  291743.218750   

           AutoETS-hi-20  AutoETS-hi-30  AutoETS-hi-40  AutoETS-hi-50  \
unique_id                                                               
Brno_1     255277.828125  257024.171875  258864.578125  260850.640625   
Brno_1     256332.828125  258628.765625  261048.328125  263659.437500   
Brno_1     257180.140625  259917.437500  262802.156250  265915.218750   
Brno_1     257908.625000  261025.421875  264310.062500  267854.687500   
Prague_1   294151.593750  296640.843750  299264.125000  302095.093750   

           AutoETS-hi-60  AutoETS-hi-70  AutoETS-hi-80  AutoETS-hi-90  \
unique_id                                                               
Brno_1      263062.25000   265640.12500   268883.68750   273691.12500   
Brno_1      266567.00000   269956.12500   274220.43750   280540.78125   
Brno_1      269381.75000   273422.40625   278506.50000   286041.87500   
Brno_1      271801.78125   276402.62500   282191.56250   290771.59375   
Prague_1    305247.46875   308922.00000   313545.34375   320397.87500   

           AutoETS-hi-95  
unique_id                 
Brno_1      277860.84375  
Brno_1      286022.71875  
Brno_1      292577.68750  
Brno_1      298213.50000  
Prague_1    326341.40625  

[5 rows x 47 columns]

In [26]:
# @title ds vs AutoARIMA

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['ds']
  ys = series['AutoARIMA']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = sf_fcst.sort_values('ds', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('unique_id')):
  _plot_series(series, series_name, i)
  fig.legend(title='unique_id', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ds')
_ = plt.ylabel('AutoARIMA')

# MLForecast

In [27]:
models = [LinearRegression(),
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    RandomForestRegressor(random_state=0),
]
@njit
def rolling_mean_3(x):
    return rolling_mean(x, window_size=3)
@njit
def rolling_mean_6(x):
    return rolling_mean(x, window_size=6)

mlf = MLForecast(
    models=models,
    freq='MS',
    lags=[1,3,5,7,12],

    lag_transforms={
        1: [expanding_mean],
        3: [rolling_mean_3, rolling_mean_6],
        5: [rolling_mean_3, rolling_mean_6],
        7: [rolling_mean_3, rolling_mean_6],
        12: [rolling_mean_3, rolling_mean_6],
    },
    date_features=['year', 'month', 'day', 'quarter'],
    target_transforms=[AutoDifferences(max_diffs=2)])


In [29]:
from mlforecast.utils import PredictionIntervals
mlf.fit(
    train,
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    static_features=[],
    prediction_intervals=PredictionIntervals(n_windows=2, h=4),
)

MLForecast(models=[LinearRegression, LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=MS, lag_features=['lag1', 'lag3', 'lag5', 'lag7', 'lag12', 'expanding_mean_lag1', 'rolling_mean_3_lag3', 'rolling_mean_6_lag3', 'rolling_mean_3_lag5', 'rolling_mean_6_lag5', 'rolling_mean_3_lag7', 'rolling_mean_6_lag7', 'rolling_mean_3_lag12', 'rolling_mean_6_lag12'], date_features=['year', 'month', 'day', 'quarter'], num_threads=1)

In [31]:
#For using in predict function
future_df = test[["unique_id", "ds", 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']]
levels=[5,10,20,30,40,50,60,70,80,90,95]

In [32]:
mlforecast = mlf.predict(h=4, X_df=future_df, level=levels)

In [33]:
mlforecast.head()

unique_id         ds  LinearRegression  LGBMRegressor  XGBRegressor  \
0    Brno_1 2023-12-01      276673.28125   262346.31250  263501.50000   
1    Brno_1 2024-01-01      289178.84375   271418.68750  273108.75000   
2    Brno_1 2024-02-01      284091.96875   270878.68750  259731.56250   
3    Brno_1 2024-03-01      322418.46875   289367.87500  276830.46875   
4  Prague_1 2023-12-01      317093.03125   309528.15625  307622.56250   

   RandomForestRegressor  LinearRegression-lo-95  LinearRegression-lo-90  \
0          264465.687500           259471.734375           260335.031250   
1          271225.875000           257288.004688           258102.978125   
2          256024.390625           245951.716797           248651.886719   
3          270088.593750           259642.812891           263268.438281   
4          303870.000000           300840.228125           302068.550000   

   LinearRegression-lo-80  LinearRegression-lo-70  ...  \
0           262061.625000           263788.218750  ...   
1           259732.925000           261362.871875  ...   
2           254052.226562           259452.566406  ...   
3           270519.689063           277770.939844  ...   
4           304525.193750           306981.837500  ...   

   RandomForestRegressor-hi-10  RandomForestRegressor-hi-20  \
0                265146.856250                265828.025000   
1                272970.084375                274714.293750   
2                257427.790625                258831.190625   
3                272557.073437                275025.553125   
4                304630.153125                305390.306250   

   RandomForestRegressor-hi-30  RandomForestRegressor-hi-40  \
0                266509.193750                267194.012500   
1                276458.503125                277408.140625   
2                260234.590625                262070.984375   
3                277494.032813                280540.007812   
4                306150.459375                307020.409375   

   RandomForestRegressor-hi-50  RandomForestRegressor-hi-60  \
0                267880.656250                268567.300000   
1                277960.492188                278512.843750   
2                264123.875000                266176.765625   
3                283874.730469                287209.453125   
4                307945.257812                308870.106250   

   RandomForestRegressor-hi-70  RandomForestRegressor-hi-80  \
0                269253.943750                269940.587500   
1                279065.195312                279617.546875   
2                268229.656250                270282.546875   
3                290544.175781                293878.898438   
4                309794.954688                310719.803125   

   RandomForestRegressor-hi-90  RandomForestRegressor-hi-95  
0                270627.231250                270970.553125  
1                280169.898438                280446.074219  
2                272335.437500                273361.882812  
3                297213.621094                298880.982422  
4                311644.651562                312107.075781  

[5 rows x 94 columns]

# Neural Forecast

In [35]:
hist_exog_list = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']
from neuralforecast.losses.pytorch import MQLoss
horizon = len(test)  # Number of future time steps to predict
# Initialize the TFT model
tft_model = TFT(h=4,  # Forecasting horizon of 12 months
                input_size=32,  # Assuming we use 12 months of history
                hist_exog_list=hist_exog_list,
                max_steps=200,
                start_padding_enabled=True,
                loss=MQLoss(level=levels))

# Initialize the NHITS model
nhits_model = NHITS(h=4,  # Forecasting horizon of 12 months
                input_size=32,  # Assuming we use 12 months of history
                hist_exog_list=hist_exog_list,
                max_steps=200,
                start_padding_enabled=True,
                loss=MQLoss(level=levels))

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [37]:
nf = NeuralForecast(models=[tft_model, nhits_model], freq='MS')  # 'M' for monthly frequency

nf.fit(df=train)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MQLoss                   | 23     | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 1.5 K  | train
4 | temporal_encoder        | TemporalCovariateEncoder | 948 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K  | train
6 | output_adapter          | Linear      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=200` reached.


In [38]:
neural_fcst = nf.predict(futr_df = future_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [39]:
neural_fcst.head()

ds     TFT-median      TFT-lo-95      TFT-lo-90  \
unique_id                                                           
Brno_1    2023-12-01  244342.109375  238200.343750  239490.515625   
Brno_1    2024-01-01  241625.859375  234697.515625  235521.812500   
Brno_1    2024-02-01  225947.921875  220153.578125  220528.046875   
Brno_1    2024-03-01  229289.578125  223141.703125  224052.984375   
Prague_1  2023-12-01  293479.812500  284738.812500  286109.062500   

               TFT-lo-80      TFT-lo-70      TFT-lo-60      TFT-lo-50  \
unique_id                                                               
Brno_1     241110.109375  240899.890625  241815.078125  242955.562500   
Brno_1     236807.296875  237073.203125  237404.437500  239916.796875   
Brno_1     221228.812500  221530.187500  222196.515625  224075.484375   
Brno_1     224829.390625  225440.640625  226122.687500  227484.656250   
Prague_1   289008.031250  289158.718750  290595.406250  291460.937500   

               TFT-lo-40      TFT-lo-30  ...   NHITS-hi-10    NHITS-hi-20  \
unique_id                                ...                                
Brno_1     242528.640625  244597.718750  ...  254032.50000  253876.546875   
Brno_1     239649.531250  240597.046875  ...  265176.59375  265879.968750   
Brno_1     224178.421875  224972.171875  ...  267429.21875  266391.656250   
Brno_1     227595.312500  229089.890625  ...  275749.09375  275246.187500   
Prague_1   292005.625000  292722.500000  ...  291725.93750  291699.906250   

            NHITS-hi-30   NHITS-hi-40    NHITS-hi-50    NHITS-hi-60  \
unique_id                                                             
Brno_1     255069.06250  255767.40625  256522.296875  257921.203125   
Brno_1     266749.75000  267535.59375  269111.937500  270053.218750   
Brno_1     268129.84375  270533.46875  268649.093750  268491.500000   
Brno_1     275037.90625  277524.40625  277075.500000  276829.218750   
Prague_1   293549.25000  293616.34375  295125.656250  296246.906250   

             NHITS-hi-70    NHITS-hi-80   NHITS-hi-90   NHITS-hi-95  
unique_id                                                            
Brno_1     258432.671875  259389.265625  260554.68750  260273.15625  
Brno_1     269898.000000  271636.468750  271282.71875  271035.25000  
Brno_1     268340.750000  271342.343750  272051.50000  273309.84375  
Brno_1     276112.156250  278748.375000  278700.81250  279172.00000  
Prague_1   296539.406250  298064.781250  299492.71875  298827.46875  

[5 rows x 47 columns]

# TimeGPT

In [40]:

# Initialize the NixtlaClient
nixtla_client = NixtlaClient(api_key='nixak-dZyIsJpKaqa6jjcPkhMCcHNI6S9heGWdO5iG2WixOCViM5afbymNqIRgE1XZHgkTx7MhzC9lhzFz4eKm')

# Assuming df is your input dataframe with historical data
# and future_df is your dataframe with future exogenous variables

# Specify the exogenous variables
exog_vars = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']
# Make the forecast
timegpt_fcst_zero = nixtla_client.forecast(df=train, h=4, X_df=future_df, freq='MS', level=levels)


In [41]:
timegpt_fcst_zero['ds'] = pd.to_datetime(timegpt_fcst_zero['ds'])

In [42]:
timegpt_fcst_zero.head()

unique_id         ds    TimeGPT  TimeGPT-hi-10  TimeGPT-hi-20  \
0    Brno_1 2023-12-01  250332.88      251943.40      253553.94   
1    Brno_1 2024-01-01  252602.64      253980.90      255359.19   
2    Brno_1 2024-02-01  235316.16      235918.03      236519.92   
3    Brno_1 2024-03-01  243055.92      243728.06      244400.19   
4  Prague_1 2023-12-01  286215.94      289442.25      292668.60   

   TimeGPT-hi-30  TimeGPT-hi-40  TimeGPT-hi-5  TimeGPT-hi-50  TimeGPT-hi-60  \
0      255164.48      256775.02     251138.14      258385.55      259996.08   
1      256737.45      258115.73     253291.77      259494.00      260872.28   
2      237121.80      237723.67     235617.10      238325.56      238927.44   
3      245072.33      245744.45     243391.98      246416.60      247088.73   
4      295894.90      299121.25     287829.10      302347.56      305573.88   

   ...  TimeGPT-lo-20  TimeGPT-lo-30  TimeGPT-lo-40  TimeGPT-lo-5  \
0  ...      247111.81      245501.27      243890.73     249527.61   
1  ...      249846.08      248467.81      247089.53     251913.50   
2  ...      234112.39      233510.52      232908.64     235015.22   
3  ...      241711.66      241039.52      240367.39     242719.86   
4  ...      279763.28      276536.97      273310.62     284602.78   

   TimeGPT-lo-50  TimeGPT-lo-60  TimeGPT-lo-70  TimeGPT-lo-80  TimeGPT-lo-90  \
0      242280.20      240669.67      239059.14      237448.60      235838.06   
1      245711.27      244332.98      242954.72      241576.44      240198.17   
2      232306.75      231704.88      231103.00      230501.12      229899.23   
3      239695.25      239023.11      238350.98      237678.84      237006.72   
4      270084.30      266858.00      263631.66      260405.34      257179.02   

   TimeGPT-lo-95  
0      235032.80  
1      239509.03  
2      229598.30  
3      236670.64  
4      255565.84  

[5 rows x 25 columns]

In [43]:
# Make the forecast
timegpt_fcst_finetune_4 = nixtla_client.forecast(df=train, h=4, X_df=future_df, freq='MS', finetune_steps=4, level=levels)

In [44]:
timegpt_fcst_finetune_6 = nixtla_client.forecast(df=train, h=4, X_df=future_df, freq='MS', finetune_steps=6, level=levels)

In [45]:
timegpt_fcst_finetune_8= nixtla_client.forecast(df=train, h=4, X_df=future_df, freq='MS', finetune_steps=8, level=levels)

In [46]:
timegpt_fcst_finetune_6.rename(columns={'TimeGPT': 'TimeGPT_6'}, inplace=True)

In [47]:
timegpt_fcst_finetune_4.rename(columns={'TimeGPT': 'TimeGPT_4'}, inplace=True)

In [48]:
timegpt_fcst_finetune_8.rename(columns={'TimeGPT': 'TimeGPT_8'}, inplace=True)

In [49]:
timegpt_fcst_finetune_4.columns = ['TimeGPT_4-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_4.columns]
timegpt_fcst_finetune_6.columns = ['TimeGPT_6-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_6.columns]
timegpt_fcst_finetune_8.columns = ['TimeGPT_8-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_8.columns]





## Timesfm

In [ ]:
# Y_test_df = test.copy()
# df = train.copy()

In [ ]:
# tfm = timesfm.TimesFm(
#     context_len=512,
#     horizon_len=8,
#     input_patch_len=32,
#     output_patch_len=128,
#     num_layers=20,
#     model_dims=1280,
#     backend="cpu",
# )
# tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

In [ ]:
# timesfm_df = tfm.forecast_on_df(
#     inputs=df,
#     freq='MS',
#     value_name='y',
#     num_jobs=-1,
# )

In [50]:
timesfm_df = pd.read_csv("timesfm_monthly_4.csv")

In [51]:
timesfm_df['ds'] = pd.to_datetime(timesfm_df['ds'])

In [52]:
#Merge All Results
data_frames = [sf_fcst, ets_tbats,mlforecast, neural_fcst,timegpt_fcst_zero,timegpt_fcst_finetune_4, timegpt_fcst_finetune_6, timegpt_fcst_finetune_8, timesfm_df ]
final_forecast_monthly_merged = reduce(lambda  left,right: pd.merge(left,right,on=['unique_id', 'ds'],how='inner'), data_frames)

In [53]:
final_forecast_monthly_merged.to_csv("Probabilistic_forecast_monthly_4_merged.csv")